In [1]:
import os
import pathlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from pprint import pprint
import numpy as np
from IPython.core.debugger import set_trace
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from skimage import io

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
this_path = pathlib.Path.cwd()

In [7]:
IMG_SIZE = 256
_mean = [0.5, 0.5, 0.5]
_std = [0.5, 0.5, 0.5]

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([IMG_SIZE, IMG_SIZE]),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std)
])

class ImageDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, transform=True):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform=transform
        self.input_dir = root_dir
        self.img_paths = []
        
        for file_path in self.input_dir.glob("train/*.jpg"):
            #file_path = str(self.input_dir.glob) + "Cat/" + file_path
            self.img_paths.append(file_path)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = io.imread(self.img_paths[idx]) 
        image = torch.from_numpy(image)
        label = 0 if "dog" in str(self.img_paths[idx]) else 1

        if self.transform:
            try:
                image = image.permute(2, 0, 1)
                image = trans(image)
            except:
                import pdb
                pdb.set_trace()

        return image, torch.Tensor([label])

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=1, bias=True)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCELoss()
transformed_dataset = ImageDataset(pathlib.Path('./data'), transform=True)
train_set, val_set = torch.utils.data.random_split(transformed_dataset, [20000, 5000])
train_dataloader = DataLoader(train_set, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=8, shuffle=True)

sigmoid = nn.Sigmoid().to(device)
model.train() # set model to training mod
for j in range(5):
    for i, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        X = batch[0]
        y = batch[1]
        X, y = X.to(device), y.to(device) # data to GPU
        y_ = model(X)
        y_ = sigmoid(y_)
        y_ = y_.reshape(y_.shape[0],1)
        loss = criterion(y_, y) 
        loss.backward(loss)
        optimizer.step()
        if i%1000 == 0:
            print('iteration:', i , 'loss: ', loss.item())
            print('iteration:', i, 'train acc:', (y_.round() == y).sum().item()/(y.shape[0]))
            # validation loss
            loss = 0
            acc = 0
            for i, batch in enumerate(val_dataloader):
                optimizer.zero_grad()
                X = batch[0]
                y = batch[1]
                X, y = X.to(device), y.to(device) # data to GPU

                y_ = model(X)
                y_ = sigmoid(y_)
                y_ = y_.reshape(y_.shape[0],1)
                loss += criterion(y_, y).item()
                acc += (y_.round() == y).sum().item()/(y.shape[0])
            print('validation loss: ', loss/(i+1))
            print('validation acc:', acc/(i+1))
            print()




iteration: 0 loss:  0.747825026512146
iteration: 0 train acc: 0.5
validation loss:  0.7292786238670349
validation acc: 0.4708

iteration: 1000 loss:  0.16089071333408356
iteration: 1000 train acc: 1.0
validation loss:  0.10791223244071006
validation acc: 0.9764

iteration: 2000 loss:  0.03702716901898384
iteration: 2000 train acc: 1.0
validation loss:  0.08869834107905626
validation acc: 0.978

iteration: 0 loss:  0.05647910013794899
iteration: 0 train acc: 1.0
validation loss:  0.07697942672520876
validation acc: 0.9782

iteration: 1000 loss:  0.05052806809544563
iteration: 1000 train acc: 1.0
validation loss:  0.08090505643337965
validation acc: 0.9826

iteration: 2000 loss:  1.3421473503112793
iteration: 2000 train acc: 0.625
validation loss:  0.06804681139588356
validation acc: 0.9832

iteration: 0 loss:  0.013626862317323685
iteration: 0 train acc: 1.0
validation loss:  0.06514666835963726
validation acc: 0.9844

iteration: 1000 loss:  0.019855575636029243
iteration: 1000 train ac

In [ ]:
"""
#saving the model
import pickle
file_to_store = open("resnet_092_all_attr_5_epochs.pkl", "wb")
pickle.dump(model, file_to_store)
file_to_store.close()

# reading the model
import pickle

file_to_read = open("resnet_092_all_attr_5_epochs.pkl", "rb")
model = pickle.load(file_to_read).to(device)
file_to_read.close()
"""